In [2]:
import os
import sys
import re
import pickle
from collections import defaultdict, OrderedDict, Counter
import warnings
warnings.filterwarnings('ignore')

import json
from tqdm import tqdm

In [5]:
import numpy as np
import pandas as pd
# from pandas.plotting import scatter_matrix
# from pandas_profiling import ProfileReport
from missingno import dendrogram
import missingno as msno

# pd.options.display.float_format = '{:0.5f}'.format`

In [6]:
import matplotlib.pyplot as plt
import seaborn as sns

# settings for seaborn plot sizes
sns.set(rc={'figure.figsize':(4.5, 3)})

In [31]:
배달_주문건수 = pd.read_csv('data/업종-지역별 배달 주문건수.csv', header=None)
평균배달소요시간 = pd.read_csv('data/업종-지역별 평균배달소요시간.csv', header=None)
평균주문금액 = pd.read_csv('data/업종-지역별 평균주문금액.csv', header=None)

In [37]:
for dff in [배달_주문건수, 평균배달소요시간, 평균주문금액]:
    dff.columns = ['날짜', '시간', '업종명', '광역시도' , '시군구', 'value']
    # print(dff.head(3))
    # print(dff.info())
    dff['group'] = dff['날짜'] + dff['시간'].astype(str) + dff['업종명'] + dff['광역시도'] + dff['시군구']

ValueError: Length mismatch: Expected axis has 7 elements, new values have 6 elements

In [49]:
df_1 = pd.merge(배달_주문건수, 평균배달소요시간, on='group', how='inner')

In [53]:
df_2 = pd.merge(df_1, 평균주문금액, on='group', how='inner')

In [48]:
for dff in [배달_주문건수, 평균배달소요시간, 평균주문금액]:
    print(len(dff['group'].unique()))

3367316
3333595
3223132


In [67]:
list(df_2.columns[-6:-1]) + ['value_x', 'value', 'value_y']

['날짜', '시간', '업종명', '광역시도', '시군구', 'value_x', 'value', 'value_y']

In [68]:
df = df_2[list(df_2.columns[-6:-1]) + ['value_x', 'value', 'value_y']]

In [69]:
df.head()

,날짜,시간,업종명,광역시도,시군구,value_x,value,value_y
0,2019-07-17,0,도시락,경기도,의정부시,1,21000,13.52
1,2019-07-17,0,돈까스/일식,경기도,의정부시,3,35000,13.02
2,2019-07-17,0,돈까스/일식,충청북도,제천시,1,42000,15.03
3,2019-07-17,0,배달전문업체,경기도,고양시 일산동구,8,23125,19.49
4,2019-07-17,0,배달전문업체,경기도,의정부시,3,16633,19.33


In [70]:
df.columns = ['날짜', '시간', '업종명',	'광역시도' , '시군구',\
                    '배달주문건수', '평균주문금액', '평균배달소요시간']

In [83]:
df.to_csv('업종지역별_주문건수_금액_배달소요시간', index=False)

----

In [71]:
import os

dir_path = "data"

path_ls = []
for (root, directories, files) in os.walk(dir_path):
    for file in files:
        file_path = os.path.join(root, file)
        path_ls.append(file_path)

In [72]:
dff_dict= defaultdict(list)
for path in tqdm(path_ls):
    name = path.split('_')[-2]
    dff = pd.read_csv(f'{path}', header=None)
    dff_dict[name].append(dff)

100%|██████████| 58/58 [01:03<00:00,  1.09s/it]


In [73]:
len(dff_dict)

14

In [74]:
concat_dff_dict = defaultdict()
for key, value in dff_dict.items():
    print(f'----------{key}----------')
    dff = pd.concat(value)
    dff.drop_duplicates(keep='first')
    concat_dff_dict[key] = (dff)

----------KGUINDTAMOUNT----------
----------KGUPOPLTNINFO----------
----------KGURESIDEINFO----------
----------KGUTRDARINFO----------
----------KGUWETHERINFO----------
----------KGUAREAAVDSTN----------
----------KGUAREADOITEM----------
----------KGUAREAINDPAY----------
----------KGUDSTNAMOUNT----------
----------KGUDSTNORDER----------
----------KGUINSCTNINFO----------
----------KGUINSCTNORDR----------
----------KGUWTHRDLVRDF----------
----------KGUDLIVERYSTORE00001----------


In [105]:
for code, dff in concat_dff_dict.items():
    print(f'{code} - {dff.shape}')

KGUINDTAMOUNT - (3223132, 6)
KGUPOPLTNINFO - (220374, 8)
KGURESIDEINFO - (495000, 9)
KGUTRDARINFO - (159651, 10)
KGUWETHERINFO - (5626750, 15)
KGUAREAAVDSTN - (14606, 7)
KGUAREADOITEM - (26398236, 6)
KGUAREAINDPAY - (11023306, 6)
KGUDSTNAMOUNT - (3794787, 6)
KGUDSTNORDER - (3794787, 6)
KGUINSCTNINFO - (30104, 23)
KGUINSCTNORDR - (54754, 7)
KGUWTHRDLVRDF - (380741, 27)
KGUDLIVERYSTORE00001 - (125766, 11)


----

In [176]:
all_dict = defaultdict(list)

In [177]:
text = '날씨별 배달 품목차이'
code = 'KGUWTHRDLVRDF'
ls = ['광역시도명',
'시군구명',
'날짜',
'시간대별 시간',
'강수 유형명',
'습도값',
'강수량 값',
'기온값',
'풍속값',
'바람강도 유형명',
'동쪽서쪽바람유형명',
'풍향값',
'풍향카테고리명',
'한식_배달건수',
'분식_배달건수',
'카페/디저트_배달건수',
'돈까스/일식_배달건수',
'회_배달건수',
'치킨_배달건수',
'피자_배달건수',
'아시안/양식_배달건수',
'중식_배달건수',
'족발/보쌈_배달건수',
'야식_배달건수',
'찜탕_배달건수',
'도시락_배달건수',
'패스트푸드_배달건수']
all_dict[text].append(code)
all_dict[text].append(ls)

In [178]:
text = '지역-업종 배달평균거리'
code = 'KGUAREAAVDSTN'
ls = ['년도',
'월',
'배달상점 업종명',
'배달상점 광역시도명',
'배달상점 시군구명',
'배달주문수',
'배달평균거리값']
all_dict[text].append(code)
all_dict[text].append(ls)

In [179]:
text = '지역-업종 결제정보'
code = 'KGUAREAINDPAY'
ls = ['날짜',
'24시기준 해당시간',
'배달상점 업종명',
'배달상점 광역시도명',
'배달상점 시군구명',
'결제종류값']
all_dict[text].append(code)
all_dict[text].append(ls)

In [180]:
text = '배달 상점 데이터'
code = 'KGUDLIVERYSTORE00001'
ls = ['배달상점ID'	,
'배달상점 업종명',
'배달상점 법정동코드'	,
'배달상점 시도명'	,
'배달상점 시군구명'	,
'배달상점 법정동 읍면동명'	,
'배달상점 법정동 리명'	,
'배달상점 행정동코드',
'배달상점 행정동 읍면동명'	,
'배달상점 도로명주소코드',
'배달상점 도로명주소 도로명']
all_dict[text].append(code)
all_dict[text].append(ls)

In [181]:
text = '업종 목적지별 배달 주문건수'
code = 'KGUDSTNORDER'
ls = ['날짜',
'시간대별 시간',
'배달상점 업종명',
'목적지 광역시도명',
'목적지 시군구명',
'주문 건수']
all_dict[text].append(code)
all_dict[text].append(ls)

In [182]:
text = '업종-거리구간 배달 건수'
code = 'KGUINSCTNORDR'
ls = ['년도',
'월',
'배달상점 업종명',
'배달상점 광역시도명',
'배달상점 시군구명',
'배달거리 구간명',
'배달주문수']
all_dict[text].append(code)
all_dict[text].append(ls)

In [183]:
text = '업종 목적지별 평균주문금액'
code = 'KGUDSTNAMOUNT'
ls = ['날짜',
'시간대별 시간',
'배달상점 업종명',
'목적지 광역시도명',
'목적지 시군구명',
'평균주문금액']
all_dict[text].append(code)
all_dict[text].append(ls)

In [184]:
text = '주문지역 주거 특성'
code = 'KGURESIDEINFO'
ls = ['기준연도',	'시군구코드',	'시도명',	'시군구명',	'세대구성대분류',	'세대구성중분류',	'세대구성소분류',	'단위','빈도수']
all_dict[text].append(code)
all_dict[text].append(ls)

In [185]:
text = '주문지역 상권유형 특성'
code = 'KGUTRDARINFO'
ls = ['기준연도',	'시도명',	'시도코드',	'시군구명'	,'시군구코드'	,'상권업종대분류명',	'상권업종중분류명'	,'상권업종소분류명'	,'상권업종소분류코드'	,'상권업종개수']
all_dict[text].append(code)
all_dict[text].append(ls)

In [186]:
text = '주문지역 인구 특성'
code = 'KGUPOPLTNINFO'
ls = ['기준연월',	'시군구코드',	'시도명',	'시군구명'	,'5세단위구분'	,'총인구수'	,'남성인구수','여성인구수']
all_dict[text].append(code)
all_dict[text].append(ls)

In [187]:
text = '주문지역 기상 정보'
code = 'KGUWETHERINFO'
ls = ['순번',	'시군구코드',	'광역시도명',	'시군구명',	'공개 일',	'공개 시간',\
    '강수 유형명',	'습도',	'1시간 강수량 값',	'기온',	'풍속',	\
    '바람강도 유형명',	'동쪽서쪽바람유형명',	'풍향값',	'풍향카테고리명']
all_dict[text].append(code)
all_dict[text].append(ls)

In [188]:
text = '지역-요일 배달 품목정보'
code = 'KGUAREADOITEM'
ls = ['날짜',
'24시기준 해당시간',
'주문요일명',
'배달상점 광역시도명',
'배달상점 시군구명',
'배달상점 업종명']
all_dict[text].append(code)
all_dict[text].append(ls)

In [189]:
text = '업종-거리구간 상권 정보'
code = 'KGUINSCTNINFO'
ls = ['년도',
'월',
'배달상점 업종명',
'배달상점 광역시도명',
'배달상점 시군구명',
'1km이하 배달건수',
'1km초과3km이하배달건수',
'3km초과5km이하배달건수',
'거리 5000이상',
'한식_매장수',
'분식_매장수',
'카페/디저트_매장수',
'돈까스/일식_매장수',
'회_매장수',
'치킨_매장수',
'피자_매장수',
'아시안/양식_매장수',
'중식_매장수',
'족발/보쌈_매장수',
'야식_매장수',
'찜탕_매장수',
'도시락_매장수',
'패스트푸드_매장수']
all_dict[text].append(code)
all_dict[text].append(ls)

In [190]:
text = '업종-지역별 평균주문금액'
code = 'KGUINDTAMOUNT'
ls = ['날짜',	'시간대별 시간',	'배달상점 업종명',	'배달상점 광역시도명',	'배달상점 시군구명',	'평균주문금액']
all_dict[text].append(code)
all_dict[text].append(ls)

- 업종 목적지별 평균주문금액 / KGUDSTNAMOUN
'날짜'
'시간대별 시간'
'배달상점 업종명'
'목적지 광역시도명'
'목적지 시군구명'
'평균주문금액'


- 업종-거리구간 배달 건수/ KGUINSCTNORDR
'년도'
'월'
'배달상점 업종명'
'배달상점 광역시도명'
'배달상점 시군구명'
'배달거리 구간명'
'배달주문수'



- 업종 목적지별 배달 주문건수 / KGUDSTNORDER
'날짜'
'시간대별 시간'
'배달상점 업종명'
'목적지 광역시도명'
'목적지 시군구명'
'주문 건수'


- 배달 상점 데이터 / KGUDLIVERYSTORE00001
'배달상점ID'	
'배달상점'
'업종명'
'배달상점 법정동코드'	
'배달상점 시도명'	
'배달상점 시군구명'	
'배달상점 법정동 읍면동명'	
'배달상점 법정동 리명'	
'배달상점 행정동코드'
'배달상점 행정동 읍면동명'	
'배달상점 도로명주소코드'	
'배달상점 도로명주소 도로명'


- 지역-업종 결제정보 / KGUAREAINDPAY
'날짜'
'24시기준 해당시간'
'배달상점 업종명'
'배달상점 광역시도명'
'배달상점 시군구명'
'결제종류값'



- 주문지역 기상 정보 / KGUWETHERINFO
'순번'	'시군구코드'	'광역시도명'	'시군구명'	'공개 일'	'공개 시간'	'강수 유형명'	'습도'	'1시간 강수량 값'	'기온'	'풍속'	'바람강도 유형명'	'동쪽서쪽바람유형명'	'풍향값'	'풍향카테고리명'




- 지역-요일 배달 품목정보 / KGUAREADOITEM
'날짜'
'24시기준 해당시간'
'주문요일명'
'배달상점 광역시도명'
'배달상점 시군구명'
'배달상점 업종명'



- 지역-업종 배달평균거리 / KGUAREAAVDSTN
'년도'
'월'
'배달상점 업종명'
'배달상점 광역시도명'
'배달상점 시군구명'
'배달주문수'
'배달평균거리값'




- 날씨별 배달 품목차이 / KGUWTHRDLVRD
'광역시도명'
'시군구명'
'날짜'
'시간대별 시간'
'강수 유형명'
'습도값'
'강수량 값'
'기온값'
'풍속값'
'바람강도 유형명'
'동쪽서쪽바람유형명'
'풍향값'
'풍향카테고리명'
'한식_배달건수'
'분식_배달건수'
'카페/디저트_배달건수'
'돈까스/일식_배달건수'
'회_배달건수'
'치킨_배달건수'
'피자_배달건수'
'아시안/양식_배달건수'
'중식_배달건수'
'족발/보쌈_배달건수'
'야식_배달건수'
'찜탕_배달건수'
'도시락_배달건수'
'패스트푸드_배달건수'


- 주문지역 주거 특성 / KGURESIDEINFO
'기준연도'	'시군구코드'	'시도명'	'시군구명'	'세대구성대분류'	'세대구성중분류'	'세대구성소분류'	'단위'	'빈도수'


- 업종-거리구간 상권 정보 / KGUINSCTNINFO
'년도'
'월'
'배달상점 업종명'
'배달상점 광역시도명'
'배달상점 시군구명'
'1km이하 배달건수'
'1km초과3km이하배달건수'
'3km초과5km이하배달건수'
'거리 5000이상'
'한식_매장수'
'분식_매장수'
'카페/디저트_매장수'
'돈까스/일식_매장수'
'회_매장수'
'치킨_매장수'
'피자_매장수'
'아시안/양식_매장수'
'중식_매장수'
'족발/보쌈_매장수'
'야식_매장수'
'찜탕_매장수'
'도시락_매장수'
'패스트푸드_매장수'


- 날씨별 배달 품목차이 / KGUWTHRDLVRDF
'광역시도명'
'시군구명'
'날짜'
'시간대별 시간'
'강수 유형명'
'습도값'
'강수량 값'
'기온값'
'풍속값'
'바람강도 유형명'
'동쪽서쪽바람유형명'
'풍향값'
'풍향카테고리명'
'한식_배달건수'
'분식_배달건수'
'카페/디저트_배달건수'
'돈까스/일식_배달건수'
'회_배달건수'
'치킨_배달건수'
'피자_배달건수'
'아시안/양식_배달건수'
'중식_배달건수'
'족발/보쌈_배달건수'
'야식_배달건수'
'찜탕_배달건수'
'도시락_배달건수'
'패스트푸드_배달건수'




- 주문지역 인구 특성 / KGUPOPLTNINFO
'기준연월'	'시군구코드'	'시도명'	'시군구명'	'5세단위구분'	'총인구수'	'남성인구수'	'여성인구수'




- 업종-지역별 평균주문금액 / KGUINDTAMOUNT
'날짜'	'시간대별 시간'	'배달상점 업종명'	'배달상점 광역시도명'	'배달상점 시군구명'	'평균주문금액'




- 주문지역 상권유형 특성 / KGUTRDARINFO
'기준연도'	'시도명'	'시도코드'	'시군구명'	'시군구코드'	'상권업종대분류명'	'상권업종중분류명'	'상권업종소분류명'	'상권업종소분류코드'	'상권업종개수'


- 주문지역 주거 특성 / KGURESIDEINFO
'기준연도'	'시군구코드'	'시도명'	'시군구명'	'세대구성대분류'	'세대구성중분류'	'세대구성소분류'	'단위'	'빈도수'


In [191]:
for code, dff in concat_dff_dict.items():
    print(f'{code} - {dff.shape}')

KGUINDTAMOUNT - (3223132, 6)
KGUPOPLTNINFO - (220374, 8)
KGURESIDEINFO - (495000, 9)
KGUTRDARINFO - (159651, 10)
KGUWETHERINFO - (5626750, 15)
KGUAREAAVDSTN - (14606, 7)
KGUAREADOITEM - (26398236, 6)
KGUAREAINDPAY - (11023306, 6)
KGUDSTNAMOUNT - (3794787, 6)
KGUDSTNORDER - (3794787, 6)
KGUINSCTNINFO - (30104, 23)
KGUINSCTNORDR - (54754, 7)
KGUWTHRDLVRDF - (380741, 27)
KGUDLIVERYSTORE00001 - (125766, 11)


In [192]:
for name, ls in all_dict.items():
    # print(name)
    code = ls[0]
    cols = ls[1]
    try:
        concat_dff_dict[code].columns = cols
    except:
        print(name)
        print(cols)
        print(concat_dff_dict[code].columns)
        print()
    # cols = ls[1]
    # concat_dff_dict[code].columns = cols
    

In [228]:
for i, (name, ls) in enumerate(all_dict.items()):
    # print(name)
    code = ls[0]
    cols = ls[1]
    print(i, name, code)
    # concat_dff_dict[code].to_csv(f'data/{name}.csv', index=False)

0 날씨별 배달 품목차이 KGUWTHRDLVRDF
1 지역-업종 배달평균거리 KGUAREAAVDSTN
2 지역-업종 결제정보 KGUAREAINDPAY
3 배달 상점 데이터 KGUDLIVERYSTORE00001
4 업종 목적지별 배달 주문건수 KGUDSTNORDER
5 업종-거리구간 배달 건수 KGUINSCTNORDR
6 업종 목적지별 평균주문금액 KGUDSTNAMOUNT
7 주문지역 주거 특성 KGURESIDEINFO
8 주문지역 상권유형 특성 KGUTRDARINFO
9 주문지역 인구 특성 KGUPOPLTNINFO
10 주문지역 기상 정보 KGUWETHERINFO
11 지역-요일 배달 품목정보 KGUAREADOITEM
12 업종-거리구간 상권 정보 KGUINSCTNINFO
13 업종-지역별 평균주문금액 KGUINDTAMOUNT


In [238]:
for i, (name, ls) in enumerate(all_dict.items()):
    # print(name)
    code = ls[0]
    cols = ls[1]
    print(i, name, code)
    for idx, col in enumerate(list(concat_dff_dict[code].columns)):
        print(f'\t{idx} {col}')
    print()
    # concat_dff_dict[code].to_csv(f'data/{name}.csv', index=False)

0 날씨별 배달 품목차이 KGUWTHRDLVRDF
	0 광역시도명
	1 시군구명
	2 날짜
	3 시간대별 시간
	4 강수 유형명
	5 습도값
	6 강수량 값
	7 기온값
	8 풍속값
	9 바람강도 유형명
	10 동쪽서쪽바람유형명
	11 풍향값
	12 풍향카테고리명
	13 한식_배달건수
	14 분식_배달건수
	15 카페/디저트_배달건수
	16 돈까스/일식_배달건수
	17 회_배달건수
	18 치킨_배달건수
	19 피자_배달건수
	20 아시안/양식_배달건수
	21 중식_배달건수
	22 족발/보쌈_배달건수
	23 야식_배달건수
	24 찜탕_배달건수
	25 도시락_배달건수
	26 패스트푸드_배달건수

1 지역-업종 배달평균거리 KGUAREAAVDSTN
	0 년도
	1 월
	2 배달상점 업종명
	3 배달상점 광역시도명
	4 배달상점 시군구명
	5 배달주문수
	6 배달평균거리값

2 지역-업종 결제정보 KGUAREAINDPAY
	0 날짜
	1 24시기준 해당시간
	2 배달상점 업종명
	3 배달상점 광역시도명
	4 배달상점 시군구명
	5 결제종류값

3 배달 상점 데이터 KGUDLIVERYSTORE00001
	0 배달상점ID
	1 배달상점 업종명
	2 배달상점 법정동코드
	3 배달상점 시도명
	4 배달상점 시군구명
	5 배달상점 법정동 읍면동명
	6 배달상점 법정동 리명
	7 배달상점 행정동코드
	8 배달상점 행정동 읍면동명
	9 배달상점 도로명주소코드
	10 배달상점 도로명주소 도로명

4 업종 목적지별 배달 주문건수 KGUDSTNORDER
	0 날짜
	1 시간대별 시간
	2 배달상점 업종명
	3 목적지 광역시도명
	4 목적지 시군구명
	5 주문 건수

5 업종-거리구간 배달 건수 KGUINSCTNORDR
	0 년도
	1 월
	2 배달상점 업종명
	3 배달상점 광역시도명
	4 배달상점 시군구명
	5 배달거리 구간명
	6 배달주문수

6 업종 목적지별 평균주문금액 KGUDSTNAMOUNT
	0 날짜
	1 시간대별 시간
	2 배달상점 업종명
	3 목적지 광역시도명
	4 목적지